# dask


In [2]:
import dask.dataframe as dd
import dask.array as da
import anndata as ad
import pandas as pd
from scipy.sparse import csr_matrix, vstack, hstack
import numpy as np
from pathlib import Path
from scipy.io import mmwrite


# File paths
obs_file_path = "cell_barcde_labels.csv"
count_matrix_file_path = "brain_atlas_full_counts_table.csv"

# Paths
obs_file_path = "./data/cell_barcode_labels.csv"
count_matrix_file_path = "./data/brain_atlas_full_counts_table.csv"
h5ad_file_path = "./data/dask_output_data.h5ad"


# Specify a larger sample size (e.g., 1e7 bytes)
sample_size = 1e7




# Use Dask to read the cell labels
cells = dd.read_csv(obs_file_path, assume_missing=True)
# cells = obs_dd.compute()



In [3]:

cells.shape


(Delayed('int-77162108-dbce-44a6-a370-a59a035ad68e'), 4)

In [4]:


# Specify a larger sample size in bytes (e.g., 10,000,000 bytes or 10 MB)
sample_size = 100000000

# Specify data types
# Assuming that the first column is object (like string) and the rest are uint8.
dtypes = {0: 'object'}
for col in range(1, 713626):  # Adjust the range based on the number of columns
    dtypes[col] = 'uint8'

# Use Dask to read the count matrix in a lazy manner with specified data types
# count_matrix_dd = dd.read_csv(count_matrix_file_path, dtype=dtypes, sample=sample_size)
# Read CSV in smaller partitions
count_matrix_dd = dd.read_csv(count_matrix_file_path, header=0,dtype=dtypes, sample=sample_size,  blocksize=1e6)  # 1MB blocks


In [ ]:
# Convert the Dask DataFrame to a Dask Array (excluding the index column)
count_matrix_da = count_matrix_dd.drop(columns=count_matrix_dd.columns[0]).values


In [ ]:
count_matrix_da = count_matrix_da.map_blocks( csr_matrix )


In [ ]:

count_matrix_da = count_matrix_da.map_blocks( csr_matrix )




In [ ]:
count_matrix_da.shape

In [4]:
adata_dict = {}

adata_dict["X"] = count_matrix_da.T 
# adata_dict["dtype"] = np.float64
# adata_dict["obsm"] = dict(
#     a=da.random.random((M, 100)),
# )
# adata_dict["layers"] = dict(
#     a=da.random.random((M, N)),
# )
adata_dict["obs"] = cells.compute()

adata_dict["var"] = count_matrix_dd['genes'].compute()



In [ ]:
adat = ad.AnnData(**adata_dict)

In [7]:
count_matrix_da.to_hdf5(h5ad_file_path, key="count_matrix", mode="w", format="table")


TypeError: Array.to_hdf5() missing 1 required positional argument: 'datapath'

In [6]:

# Compute the Dask array and convert it to a CSR sparse matrix
sparse_matrix_csr = csr_matrix(count_matrix_da.compute())


KeyboardInterrupt: 

In [ ]:

# Write the CSR matrix to MTX format
mmwrite("output_matrix_csr.mtx", sparse_matrix_csr)

In [15]:
# Compute the Dask array and convert it to a COO sparse matrix
sparse_matrix = coo_matrix(data.compute())

# Write the sparse matrix to MTX format
mmwrite("output_matrix.mtx", sparse_matrix)

KeyboardInterrupt: 

In [12]:
var_=pd.DataFrame(index=genes[:100])


KeyboardInterrupt: 

In [ ]:


def append_chunk_to_h5ad(adata_chunk, h5ad_path):
    """Appends the chunk of data to the existing .h5ad file."""
    
    if h5ad_path.exists(): # os.path.exists(h5ad_path):
        adata_existing = anndata.read(h5ad_path)
        adata_combined = adata_existing.concatenate(adata_chunk, index_unique=None)
        adata_combined.write(h5ad_path)
    else:
        adata_chunk.write(h5ad_path)



In [ ]:


# Create an empty list to store sparse matrices from each chunk
sparse_matrices = []
genes = []
chunk_counter = 0
st = time.time()
# 1. Read the CSV in chunks using a context manager
with pd.read_csv(count_matrix_file_path, chunksize=chunk_size) as reader:
    for chunk in reader:
        chunk_counter += 1
        print(f"chunk number {chunk_counter}")
        # 2. get the genes
        gene_chunk = chunk['genes']
        genes.append(gene_chunk)
        # 3. Convert the chunk to a sparse matrix and store
        sparse_chunk = csr_matrix(chunk.iloc[:,1:].values, dtype=np.uint8)
        sparse_matrices.append(sparse_chunk)

        lt = time.time() - st
        print(f'read #{chunk_counter}x  = {chunk.shape} in {lt:2f}s')

        st = time.time()


# 4. Combine the sparse matrices vertically
sparse_matrix = vstack(sparse_matrices)
genes_ = pd.concat(genes, axis=0)

adata = anndata.AnnData(X=sparse_matrix.transpose(),
                                obs=obs_,
                                var=pd.DataFrame(index=genes_))





In [ ]:
X=count_matrix_da.compute()


In [ ]:

# Create an AnnData object using Dask arrays
adata = anndata.AnnData(X=count_matrix_da.compute(), obs=cells, var=var_)

print(adata)
adata.write(h5ad_file_path)


In [ ]:
count_matrix_da.shape

In [ ]:
genes.shape

In [ ]:
import dask.dataframe as dd
import numpy as np
import anndata

# File paths
obs_file_path = "cell_barcde_labels.csv"
count_matrix_file_path = "brain_atlas_full_counts_table.csv"

# Specify a larger sample size in bytes (e.g., 10,000,000 bytes or 10 MB)
sample_size = 10000000

# Specify data types
# Assuming that the first column is object (like string) and the rest are uint8.
dtypes = {0: 'object'}
for col in range(1, 71001):  # Adjust the range based on the number of columns
    dtypes[col] = 'uint8'

# Use Dask to read the count matrix in a lazy manner with specified data types
count_matrix_dd = dd.read_csv(count_matrix_file_path, dtype=dtypes, sample=sample_size)
genes = count_matrix_dd.columns[1:]

# Use Dask to read the cell labels (assuming they are strings)
obs_dd = dd.read_csv(obs_file_path, dtype=str, sample=sample_size)
cells = obs_dd.compute()

# Create an AnnData object
# Compute the Dask DataFrame to get a Pandas DataFrame, then convert to numpy array
adata = anndata.AnnData(X=count_matrix_dd.compute().values, 
                        obs=cells, 
                        var=pd.DataFrame(index=genes))

print(adata)
